# Artificial Neural Network

In [370]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import joblib

In [312]:
tf.__version__

'2.3.0'

In [313]:
data = pd.read_csv("train.csv")

In [314]:
data.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,0.0,2
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,NaN,Brown,0.15,40.90,15,4,2.0,4
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,0.0,2
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,0.0,1


In [315]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18834 entries, 0 to 18833
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pet_id          18834 non-null  object 
 1   issue_date      18834 non-null  object 
 2   listing_date    18834 non-null  object 
 3   condition       17357 non-null  float64
 4   color_type      18834 non-null  object 
 5   length(m)       18834 non-null  float64
 6   height(cm)      18834 non-null  float64
 7   X1              18834 non-null  int64  
 8   X2              18834 non-null  int64  
 9   breed_category  18834 non-null  float64
 10  pet_category    18834 non-null  int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 1.6+ MB


## Impute Missing Data

In [316]:
data["condition"].value_counts()

1.0    6819
0.0    6281
2.0    4257
Name: condition, dtype: int64

In [317]:
data["condition"] = data["condition"].fillna(3.0)

In [318]:
data.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,0.0,2
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,3.0,Brown,0.15,40.90,15,4,2.0,4
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,0.0,2
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,0.0,1


In [319]:
data["condition"].value_counts()

1.0    6819
0.0    6281
2.0    4257
3.0    1477
Name: condition, dtype: int64

## Encode Data

### Encode Independent variable

In [320]:
#le = LabelEncoder()
#data["color_type"] = le.fit_transform(data["color_type"])

In [321]:
data.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,0.0,2
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,3.0,Brown,0.15,40.90,15,4,2.0,4
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,0.0,2
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,0.0,1


In [322]:
color_type_enc = OneHotEncoder()
color_type_1h = color_type_enc.fit_transform(data[["color_type"]])

In [323]:
X = data[["condition", "length(m)", "height(cm)", "X1", "X2"]].values

In [324]:
X = np.concatenate((X,color_type_1h.toarray()), axis=1)

In [339]:
print(X)
print(X.shape)

[[ 2.    0.8   7.78 ...  0.    0.    0.  ]
 [ 1.    0.72 14.19 ...  1.    0.    0.  ]
 [ 3.    0.15 40.9  ...  0.    0.    0.  ]
 ...
 [ 0.    0.99 28.13 ...  0.    0.    0.  ]
 [ 0.    0.55 44.82 ...  0.    0.    0.  ]
 [ 0.    0.86 37.4  ...  0.    0.    0.  ]]
(18834, 61)


### Encode Dependent variable

In [340]:
enc_y1 = OneHotEncoder()
y = enc_y1.fit_transform(data[["breed_category"]]).toarray()

In [341]:
print(y)

[[1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


## Split Train and Test Set

In [345]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

## Scale Data

In [346]:
sc_X = StandardScaler()
#sc_y = StandardScaler()
X_train[:,:5] = sc_X.fit_transform(X_train[:,:5])
#y_train = sc_y.fit_transform(y_train)

In [347]:
print(X_train)

[[-0.05587848  0.78685575 -0.80279881 ...  0.          0.
   0.        ]
 [-1.1297573   1.13345267  0.50396968 ...  0.          0.
   0.        ]
 [ 2.09187917 -0.98078854  0.88523861 ...  0.          0.
   0.        ]
 ...
 [ 1.01800035  1.51470928  0.31102916 ...  0.          0.
   0.        ]
 [-1.1297573  -1.46602423  0.60927985 ...  0.          0.
   0.        ]
 [ 2.09187917 -0.35691408 -0.58679767 ...  0.          0.
   0.        ]]


In [348]:
print(y_train)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


## Build the ANN for Breed Category

In [349]:
ann = tf.keras.models.Sequential()

In [350]:
ann.add(tf.keras.layers.Dense(units=64, activation='relu'))

In [351]:
ann.add(tf.keras.layers.Dense(units=64, activation='relu'))

In [352]:
ann.add(tf.keras.layers.Dense(units=64, activation='relu'))

In [353]:
ann.add(tf.keras.layers.Dense(units=3,activation='softmax'))

In [354]:
ann.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [355]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
471/471 [==============================] - 1s 1ms/step - loss: 0.2990 - categorical_accuracy: 0.8598
Epoch 2/100
471/471 [==============================] - 1s 1ms/step - loss: 0.1570 - categorical_accuracy: 0.9004
Epoch 3/100
471/471 [==============================] - 1s 1ms/step - loss: 0.1534 - categorical_accuracy: 0.9011
Epoch 4/100
471/471 [==============================] - 1s 1ms/step - loss: 0.1505 - categorical_accuracy: 0.9035
Epoch 5/100
471/471 [==============================] - 1s 1ms/step - loss: 0.1487 - categorical_accuracy: 0.9048
Epoch 6/100
471/471 [==============================] - 1s 1ms/step - loss: 0.1465 - categorical_accuracy: 0.9073
Epoch 7/100
471/471 [==============================] - 1s 1ms/step - loss: 0.1466 - categorical_accuracy: 0.9061
Epoch 8/100
471/471 [==============================] - 1s 1ms/step - loss: 0.1457 - categorical_accuracy: 0.9069
Epoch 9/100
471/471 [==============================] - 1s 1ms/step - loss: 0.1440 - categorical_

In [357]:
X_test[:,:5] = sc_X.transform(X_test[:,:5])

In [362]:
y_pred = ann.predict(X_test)
print(y_pred)

[[1.00000000e+00 1.91509881e-22 0.00000000e+00]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00]
 [9.90512550e-01 9.48741939e-03 1.61775127e-08]
 ...
 [0.00000000e+00 1.00000000e+00 0.00000000e+00]
 [7.28043830e-08 5.60059879e-15 9.99999881e-01]
 [1.07526015e-16 1.29327399e-27 1.00000000e+00]]


In [359]:
#y_pred = sc_y.inverse_transform(y_pred)
#print(y_pred)

[[0.9789672  0.4434194  0.07718856]
 [0.47939205 0.9402077  0.07718856]
 [0.9742275  0.44813263 0.07718856]
 ...
 [0.47939205 0.9402077  0.07718856]
 [0.47939208 0.4434194  0.34407893]
 [0.47939205 0.4434194  0.34407896]]


In [363]:
np.concatenate((enc_y1.inverse_transform(y_pred), enc_y1.inverse_transform(y_test)), axis =1)

array([[0., 0.],
       [1., 1.],
       [0., 1.],
       ...,
       [1., 1.],
       [2., 2.],
       [2., 2.]])

In [364]:
confusion_matrix(enc_y1.inverse_transform(y_test), enc_y1.inverse_transform(y_pred))

array([[1650,  127,    0],
       [ 245, 1431,    0],
       [   0,    0,  314]])

In [368]:
f1_score(enc_y1.inverse_transform(y_test), enc_y1.inverse_transform(y_pred), average='weighted')

0.9010327799982764

In [374]:
ann.save("breed_category_ann.h5")

## Build the ANN for Pet Category